In [1]:
import cv2
cap=cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError('')
while True:
    ret, frame = cap.read()
    frame=cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    cv2.imshow('Input', frame)
    
    c=cv2.waitKey(0)
    
    if c==30:
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2 
import numpy as np 

def update_pts(params, x, y):
    global x_init, y_init
    params["top_left_pt"] = (min(x_init, x), min(y_init, y))
    params["bottom_right_pt"] = (max(x_init, x), max(y_init, y))
    img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x]
 
def draw_rectangle(event, x, y, flags, params): 
    global x_init, y_init, drawing
    # First click initialize the init rectangle point 
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True 
        x_init, y_init = x, y 
    # Meanwhile mouse button is pressed, update diagonal rectangle point 
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        update_pts(params, x, y) 
    # Once mouse botton is release
    elif event == cv2.EVENT_LBUTTONUP: 
        drawing = False 
        update_pts(params, x, y) 


In [3]:
    drawing = False 
    event_params = {"top_left_pt": (-1, -1), "bottom_right_pt": (-1, -1)}
 
    cap = cv2.VideoCapture(0) 
    cap.set(CV_CAP_PROP_FRAME_WIDTH,640);
    cap.set(CV_CAP_PROP_FRAME_HEIGHT,480);
 
    # Check if the webcam is opened correctly 
    if not cap.isOpened(): 
        raise IOError("Cannot open webcam") 
 
    cv2.namedWindow('Webcam')
    # Bind draw_rectangle function to every mouse event
    cv2.setMouseCallback('Webcam', draw_rectangle, event_params) 
 
    while True: 
        ret, frame = cap.read() 
        img = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA) 
        (x0,y0), (x1,y1) = event_params["top_left_pt"], event_params["bottom_right_pt"]
        img[y0:y1, x0:x1] = 255 - img[y0:y1, x0:x1] 
        cv2.imshow('Webcam', img) 
 
        c = cv2.waitKey(1) 
        if c == 27: 
            break 
 
    cap.release() 
    cv2.destroyAllWindows() 

NameError: name 'CV_CAP_PROP_FRAME_WIDTH' is not defined

In [4]:
import cv2 
import numpy as np 

def cartoonize_image(img, ksize=5, sketch_mode=False):
    num_repetitions, sigma_color, sigma_space, ds_factor = 10, 5, 7, 4 
    # Convert image to grayscale 
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

    # Apply median filter to the grayscale image 
    img_gray = cv2.medianBlur(img_gray, 7) 

    # Detect edges in the image and threshold it 
    edges = cv2.Laplacian(img_gray, cv2.CV_8U, ksize=ksize) 
    ret, mask = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY_INV) 

    # 'mask' is the sketch of the image 
    if sketch_mode: 
        return cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) 

    # Resize the image to a smaller size for faster computation 
    img_small = cv2.resize(img, None, fx=1.0/ds_factor, fy=1.0/ds_factor, interpolation=cv2.INTER_AREA)

    # Apply bilateral filter the image multiple times 
    for i in range(num_repetitions): 
        img_small = cv2.bilateralFilter(img_small, ksize, sigma_color, sigma_space) 

    img_output = cv2.resize(img_small, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_LINEAR) 

    dst = np.zeros(img_gray.shape) 

    # Add the thick boundary lines to the image using 'AND' operator 
    dst = cv2.bitwise_and(img_output, img_output, mask=mask) 
    return dst 


In [5]:
    cap = cv2.VideoCapture(0) 

    cur_mode = None
    while True: 
        ret, frame = cap.read() 
        frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA) 

        c = cv2.waitKey(1) 
        if c == 27: 
            break 

        if c != -1 and c != 255 and c != cur_mode: 
            cur_mode = c 

        if cur_mode == ord('s'): 
            cv2.imshow('Cartoonize', cartoonize_image(frame, ksize=5, sketch_mode=True)) 
        elif cur_mode == ord('c'): 
            cv2.imshow('Cartoonize', cartoonize_image(frame, ksize=5, sketch_mode=False)) 
        else: 
            cv2.imshow('Cartoonize', frame) 

    cap.release() 
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [2]:
import cv2 
import numpy as np 
 
face_cascade = cv2.CascadeClassifier('../cascade_files/haarcascade_frontalface_alt.xml') 
 
cap = cv2.VideoCapture(0) 
scaling_factor = 0.5 
 
while True: 
    ret, frame = cap.read() 
    frame = cv2.resize(frame, None, fx=scaling_factor, 
 fy=scaling_factor, interpolation=cv2.INTER_AREA)
 
    face_rects = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=3) 
    for (x,y,w,h) in face_rects: 
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3) 
 
    cv2.imshow('Face Detector', frame) 
 
    c = cv2.waitKey(1) 
    if c == 27: 
        break 
 
cap.release() 
cv2.destroyAllWindows()

error: /io/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize


In [5]:
import cv2 
import numpy as np 

mouth_cascade = cv2.CascadeClassifier('../cascade_files/haarcascade_mcs_mouth.xml') 

moustache_mask = cv2.imread('../images/moustache.png')
h_mask, w_mask = moustache_mask.shape[:2] 

if mouth_cascade.empty(): 
  raise IOError('Unable to load the mouth cascade classifier xml file') 

cap = cv2.VideoCapture(0) 
scaling_factor = 0.5 

while True: 
    ret, frame = cap.read() 
    frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA) 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 

    mouth_rects = mouth_cascade.detectMultiScale(gray, 1.3, 5) 
    if len(mouth_rects) > 0: 
        (x,y,w,h) = mouth_rects[0] 
        h, w = int(0.6*h), int(1.2*w) 
        x -= int(0.05*w)
        y -= int(0.55*h)
        frame_roi = frame[y:y+h, x:x+w] 
        moustache_mask_small = cv2.resize(moustache_mask, (w, h), interpolation=cv2.INTER_AREA) 

        gray_mask = cv2.cvtColor(moustache_mask_small, cv2.COLOR_BGR2GRAY) 
        ret, mask = cv2.threshold(gray_mask, 50, 255, cv2.THRESH_BINARY_INV) 
        mask_inv = cv2.bitwise_not(mask) 
        masked_mouth = cv2.bitwise_and(moustache_mask_small, moustache_mask_small, mask=mask) 
        masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv) 
        frame[y:y+h, x:x+w] = cv2.add(masked_mouth, masked_frame) 

    cv2.imshow('Moustache', frame)
    c = cv2.waitKey(1) 
    if c == 27: 
        break 

cap.release() 
cv2.destroyAllWindows()

error: /io/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize
